# 1. Import all the packages we need

In [1]:
import pandas as pd
import numpy as np 
import time
import pickle
import sys
import os

# 2. Run the following code to make sure all the functions can work

In [9]:
def outlier_finder(data):
    """
    find outliers in the data
    return the indexes of the outliers
    """
    if not isinstance(data, np.ndarray):
        data = np.array(data)
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    upper_bound = q3 + 1.5 * (q3 - q1)
    lower_bound = q1 - 1.5 * (q3 - q1)
    outliers = []
    for index, item in enumerate(data):
        if item > upper_bound or item < lower_bound:
            outliers.append(index)
    return outliers

def replace_val(data, outliers, mode='max'):
    """
    modify the outliers by the mean or max value of the rest data
    """
    data_list = []
    for i in range(len(data)):
        if i not in outliers:
            data_list.append(data[i])
    if mode == 'max':
        return np.max(data_list)
    elif mode == 'mean':
        return np.mean(data_list)
    else:
        print ('Node such mode')

class ProgressBar:
    """
    make a progress bar to show the progress
    """
    def __init__(self, count = 0, total = 0, width = 50):
        self.count = count
        self.total = total
        self.width = width
        self.progress = 0
    
    def move(self):
        self.count += 1
    
    def log(self):
        new_progress = self.width * self.count // self.total
        if new_progress > self.progress:
            sys.stdout.write(' ' * (self.width + 9) + '\r')
            sys.stdout.flush()
            self.progress = new_progress
            sys.stdout.write('{0:3}/{1:3}: '.format(self.count, self.total))
            sys.stdout.write('#' * self.progress + '-' * int(self.width - self.progress) + '\r')
            if self.progress == self.width:
                sys.stdout.write('\n')
            sys.stdout.flush()

def execute_func(input_file, output_file, mode='max'):
    """
    modify all the outliers
    """
    df = pd.read_csv(input_file)
    cols = ['predict_' + str(i) for i in range (1, 11, 1)]
    df_predict_vals = df[cols]

    t1 = time.time()
    d ={}
    bar = ProgressBar(total = df_predict_vals.shape[0])
    for index, row in df_predict_vals.iterrows():
        outliers = outlier_finder( row )
        if outliers:
            new_val = replace_val(row, outliers, mode)
            for outlier in outliers:
                row[outlier] = new_val
            d[index] = outliers
        bar.move()
        bar.log()
    t2 = time.time()
    print ('cost time {0:.2f}min'.format((t2 - t1) / 60))
    print ('{}/{} rows are modified'.format(len(d), df_predict_vals.shape[0]))

    df_features = pd.concat([df[['xid', 'yid', 'hour', 'real']], df_predict_vals], axis=1)
    df_features.to_csv(output_file, index=False)
    del df, df_predict_vals, df_features

# 3. Split the data into 5 files according to the days

In [7]:
# dataset directory
file_path = '../dataset'

# split the data according to the days
for i in range(5):
    time_start = time.time()
    
    # file name
    file_name = 'ForecastDataforTraining_day' + str(i + 1) + '.csv'
    
    file_list = [x[2] for x in os.walk(file_path)][0]
    if file_name in file_list:
        print ('{} already exists'.format(file_name))
        continue
    
    print ('start Day {}'.format(i + 1))
    
    # read the data in chunk
    df_predicting_train = pd.read_csv('../dataset/ForecastDataforTraining_201802.csv', chunksize=1e7)
    
    df = pd.DataFrame()
    for chunk in df_predicting_train:
        df = pd.concat([df, chunk[chunk.date_id == i + 1]])
    
    # store data
    df.to_csv(os.path.join(file_path, file_name), index=False)
    del df
    
    time_end = time.time()
    print ('Day {0} done! cost {1:.2f} min'.format(i + 1, (time_end - time_start) / 60.0))

start Day 1
Day 1 done! cost 6.19 min
start Day 2
Day 2 done! cost 5.78 min
start Day 3
Day 3 done! cost 5.58 min
start Day 4
Day 4 done! cost 5.52 min
start Day 5
Day 5 done! cost 5.49 min


### Let's have a look at the data

In [8]:
day = 1
df = pd.read_csv('../dataset/ForecastDataforTraining_day' + str(day) + '.csv')
df.head()

,xid,yid,date_id,hour,model,wind,rainfall
0,1,1,1,3,1,9.76,0.0
1,1,1,1,3,2,9.97,0.0
2,1,1,1,3,3,8.93,0.0
3,1,1,1,3,4,9.88,0.0
4,1,1,1,3,5,9.09,0.0


# 4. Change the data into the format we want

### Emmm... we split the data into 5 parts so that we can handle the data on our laptop or there may be a memory error.

### However, this is still not what we want.

### We want the columns to look like [xid, yid, hour, model_1, model_2, ..., model_10, real].

### Also we want the wind and rainfall in different files.

### So we wrote the following code.

### The order of the rows must be the same as the original data or there may be a mistake

In [3]:
def find_index_by_xyh(xid, yid, hour):
    return (hour-3)*548*421 + (xid-1)*421 + yid - 1

def replace_index(index):
    xyh = index.split('_')
    return find_index_by_xyh(int(xyh[0]), int(xyh[1]), int(xyh[2]))


if not os.path.exists('../dataset/x_y_hour.pickle'):
    t1 = time.time()
    df = pd.read_csv('../dataset/ForecastDataforTraining_day1.csv')
    df['x_y_hour'] = df['xid'].astype(int).astype(str) + '_' + df['yid'].astype(int).astype(str)  + '_' + df['hour'].astype(int).astype(str) 
    with open('../dataset/x_y_hour.pickle', 'wb') as f:
        pickle.dump(df['x_y_hour'], f)
    t2 = time.time()
    print ('cost {0:.2f}min to get x_y_hour.pickle'.format((t2 - t1) / 60))
    del df

for day in range(1, 6, 1):
    for mode in ['wind', 'rainfall']:
        print ('start day {}, mode {}'.format(day, mode))
        if os.path.exists('../dataset/' + mode + '_data_day' + str(day) + '.csv'):
            print (mode + '_data_day' + str(day) + '.csv is already there' )
            continue
        t1 = time.time()
        df = pd.read_csv('../dataset/ForecastDataforTraining_day' + str(day) + '.csv')
        with open('../dataset/x_y_hour.pickle', 'rb') as f:
            x_y_hour = pickle.load(f)
        df['x_y_hour'] = x_y_hour
        df = df.pivot('x_y_hour', 'model', mode)
        df['x_y_hour'] = df.index
        df['real_index'] = df['x_y_hour'].apply(replace_index)
        df = df.sort_values(by=['real_index'])
        df = df.reset_index(drop=True)
        df_real = pd.read_csv('../dataset/In_situMeasurementforTraining_201802.csv')
        df_real_day = df_real[df_real.date_id == day].reset_index(drop=True)
        del df_real
        cols = [i for i in range(1, 11)]
        cols_for_real = ['xid', 'yid', 'hour', mode]
        df = pd.concat([df[cols], df_real_day[cols_for_real]], axis=1)
        del df_real_day
        cols_for_real[-1] = 'real'
        features = ['predict_' + str(i) for i in range(1, 11)] + cols_for_real
        df.columns = features
        df.to_csv('../dataset/' + mode + '_data_day' + str(day)+ '.csv', index=False)
        del df
        t2 = time.time()
        print ('cost {0:.2f}min'.format((t2 - t1) / 60))

cost 6.56min to get x_y_hour.pickle
start day 1, mode wind
cost 2.59min
start day 1, mode rainfall
cost 2.69min
start day 2, mode wind
cost 2.99min
start day 2, mode rainfall
cost 3.40min
start day 3, mode wind
cost 4.45min
start day 3, mode rainfall
cost 3.44min
start day 4, mode wind
cost 5.00min
start day 4, mode rainfall
cost 7.03min
start day 5, mode wind
cost 4.81min
start day 5, mode rainfall
cost 5.12min


### Let's see what we got

In [4]:
day = 1
df = pd.read_csv('../dataset/wind_data_day' + str(day) + '.csv')
df.head()

,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9,predict_10,xid,yid,hour,real
0,9.76,9.97,8.93,9.88,9.09,9.68,8.22,9.18,8.79,9.40,1,1,3,9.27
1,9.61,9.95,9.01,10.00,9.09,9.63,8.38,9.30,8.91,9.29,1,2,3,9.24
2,9.58,9.83,8.98,10.10,9.09,9.65,8.53,9.42,8.94,9.17,1,3,3,9.24
3,9.45,9.83,8.94,10.10,9.21,9.72,8.77,9.51,9.06,9.10,1,4,3,9.24
4,9.45,9.73,8.82,10.20,9.24,9.72,8.93,9.49,9.18,9.10,1,5,3,9.34


In [6]:
day = 1
df = pd.read_csv('../dataset/rainfall_data_day' + str(day) + '.csv')
df.head()

,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9,predict_10,xid,yid,hour,real
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,3,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2,3,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3,3,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,4,3,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,5,3,0.0


# 5. Outlier Processing

### There are some strange values in the data (extremely small or big). We don't want them.

### So replace them with the max or mean value of the rest(normal) data

In [10]:
for day in range(1, 6, 1):
    for mode in ['wind', 'rainfall']:
        print ('start day {} mode {}'.format(day, mode))
        input_file = '../dataset/' + mode + '_data_day' + str(day) + '.csv'
        output_file = '../dataset/' + mode + '_data_day' + str(day) + '_max1.csv'
        execute_func(input_file, output_file, mode='max')

start day 1 mode wind
4152744/4152744: ##################################################
cost time 17.07min
1286873/4152744 rows are modified
start day 1 mode rainfall
4152744/4152744: ##################################################
cost time 18.28min
1814255/4152744 rows are modified
start day 2 mode wind
4152744/4152744: ##################################################
cost time 17.29min
1389295/4152744 rows are modified
start day 2 mode rainfall
4152744/4152744: ##################################################
cost time 19.16min
1750353/4152744 rows are modified
start day 3 mode wind
4152744/4152744: ##################################################
cost time 16.98min
1293340/4152744 rows are modified
start day 3 mode rainfall
4152744/4152744: ##################################################
cost time 14.44min
454687/4152744 rows are modified
start day 4 mode wind
4152744/4152744: ##################################################
cost time 16.97min
1242795/4152744 rows a